<a href="https://colab.research.google.com/github/ysMarcos/projeto-python/blob/main/projeto_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports das Bibliotecas usadas

In [ ]:
#Imports
from google.colab import userdata


b4718540b964d64427a2e5434eda6274
